In [1]:
from tf_idf_my_func import *

In [2]:
import pandas as pd
from rouge import Rouge

In [3]:
dataset = load_dataset("billsum")

Found cached dataset billsum (/Users/emilygong/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
train = pd.DataFrame(dataset['train'] )
train = train.dropna()
test = pd.DataFrame(dataset['test'] )
test = test.dropna()
ca_test = pd.DataFrame(dataset['ca_test'] )
ca_test = ca_test.dropna()

# Preprocessing test dataset

In [5]:
example = test.iat[0, 0]

In [6]:
example_sentences = split_right(example)

In [7]:
cleaned_sentences = []
for i in range(len(example_sentences)):
    cleaned_sentences.append(preprocessing(example_sentences[i]))

In [8]:
for i in range(len(cleaned_sentences)):
    cleaned_sentences[i] = delete_leading_white_spaces(cleaned_sentences[i])

In [9]:
stemmed_sentences = stemming(cleaned_sentences)

# TF-IDF

## For each document, create a frequency matrix

In [10]:
freq_matrix = create_freq_matrix(example_sentences, stemmed_sentences)

## Term Frequency TF
TF(t) = (Number of times term t appears in a sentence) / (Total number of terms in the sentence)

In [11]:
tf_matrix = tf(freq_matrix)

## Inverse Document Frequency IDF

First, we calculate, “how many sentences contain a word”, Let’s call it Documents per words matrix.

Then, we apply the formula IDF(t) = log_e(Total number of sentences / Number of sentences with term t in it)

In [12]:
nums_sent_per_word = num_sent_per_word(stemmed_sentences)

In [13]:
idf_matrix = idf(freq_matrix, nums_sent_per_word, len(stemmed_sentences))

## TF-IDF

For the tf and idf values associated with each stem, multiply tf * idf

In [14]:
tf_idf_matrix = tf_idf(tf_matrix, idf_matrix)

## Score the Sentences & Find Avg Score

In [15]:
sentences_score = score_sentences(tf_idf_matrix)

In [16]:
threshold = average_sent_score(sentences_score)

# Generate Summary

In [17]:
summary = generate_summary(example_sentences, sentences_score, threshold)

# Everythin in 1 function

In [18]:
final = everything_generate_summary(example, 1)

# Evaluate the Summary with Rouge

In [19]:
rouge = Rouge()

scores = rouge.get_scores(final, example)

In [20]:
scores

[{'rouge-1': {'r': 0.2032520325203252, 'p': 1.0, 'f': 0.33783783503013154},
  'rouge-2': {'r': 0.14309210526315788,
   'p': 0.8787878787878788,
   'f': 0.2461103229098388},
  'rouge-l': {'r': 0.2032520325203252, 'p': 1.0, 'f': 0.33783783503013154}}]

# Reference

* https://towardsdatascience.com/text-summarization-using-tf-idf-e64a0644ace3
* https://en.wikipedia.org/wiki/Tf%E2%80%93idf